# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityDF = pd.read_csv("city_data.csv")
cityDF.head(10)

,Unnamed: 0,city_name,country,date,lat,long,max_temp,humidity,cloudiness,wind_speed
0,0,Marystown,CA,1590112748,47.17,-55.15,44.58,74,100,18.14
1,1,Punta Arenas,CL,1590112749,-53.15,-70.92,41.00,80,75,11.41
2,2,Bosaso,SO,1590112752,11.28,49.18,90.00,50,0,8.93
3,3,Quelimane,MZ,1590112753,-17.88,36.89,64.40,93,20,3.36
4,4,Bredasdorp,ZA,1590112754,-34.53,20.04,53.60,81,20,3.36
5,5,Hermanus,ZA,1590112755,-34.42,19.23,60.12,75,11,2.89
6,6,Departamento de Maldonado,UY,1590112756,-34.67,-54.92,62.01,100,90,9.17
7,7,Srednekolymsk,RU,1590112757,67.45,153.68,37.99,69,92,10.58
8,8,Cochrane,CA,1590112758,51.18,-114.47,41.00,86,90,17.38
9,9,Airai,TL,1590112759,-8.93,125.41,67.06,100,96,2.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
#locations
locations = cityDF[["lat", "long"]]

#humidity
humidity = cityDF["humidity"]
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations,
                                    weights=humidity,
                                    dissipating=False,
                                    max_intensity=300,
                                    point_radius=5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = cityDF.loc[(cityDF['max_temp'] > 70) &
                           (cityDF['max_temp'] < 80) &
                           (cityDF['wind_speed'] < 10) &
                           (cityDF['cloudiness'] == 0)]
hotel_df = hotel_df.dropna()
hotel_df

,Unnamed: 0,city_name,country,date,lat,long,max_temp,humidity,cloudiness,wind_speed
108,108,Riyadh,SA,1590112762,24.69,46.72,75.20,23,0,2.24
157,157,Kiryat Gat,IL,1590112928,31.61,34.76,78.01,33,0,1.01
236,236,Teya,RU,1590113013,60.38,92.63,72.61,27,0,3.31
241,241,Baghdad,IQ,1590112854,33.34,44.40,73.40,38,0,3.36
303,303,São Félix do Xingu,BR,1590112836,-6.64,-51.99,72.84,92,0,1.90
305,305,Buraidah,SA,1590112983,26.33,43.98,73.40,35,0,6.93
331,331,Dhārchula,IN,1590113120,29.85,80.53,77.34,29,0,1.52
398,398,Dawlatabad,AF,1590113196,36.41,64.91,73.24,35,0,3.44
406,406,Rio Brilhante,BR,1590113207,-21.80,-54.55,73.49,78,0,7.27


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = np.nan
for index, row in hotel_df.iterrows():
    params = {"location": f"{row['lat']}, {row['long']}",
              "keyword": 'hotel',
              "radius": 5000,
              "key": g_key}
    response = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json",
                            params=params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]["name"]
    except:
        pass
    
hotel_df = hotel_df.dropna()
hotel_df

,Unnamed: 0,city_name,country,date,lat,long,max_temp,humidity,cloudiness,wind_speed,Hotel Name
108,108,Riyadh,SA,1590112762,24.69,46.72,75.20,23,0,2.24,Four Seasons Hotel Riyadh At Kingdom Center
157,157,Kiryat Gat,IL,1590112928,31.61,34.76,78.01,33,0,1.01,Desert Gat
236,236,Teya,RU,1590113013,60.38,92.63,72.61,27,0,3.31,NaN
241,241,Baghdad,IQ,1590112854,33.34,44.40,73.40,38,0,3.36,Baghdad Hotel
303,303,São Félix do Xingu,BR,1590112836,-6.64,-51.99,72.84,92,0,1.90,Dumont Palace Hotel
305,305,Buraidah,SA,1590112983,26.33,43.98,73.40,35,0,6.93,YARA SUITES
331,331,Dhārchula,IN,1590113120,29.85,80.53,77.34,29,0,1.52,Hotel new yash
398,398,Dawlatabad,AF,1590113196,36.41,64.91,73.24,35,0,3.44,NaN
406,406,Rio Brilhante,BR,1590113207,-21.80,-54.55,73.49,78,0,7.27,Gold Hotel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))